# Application Streamlit via Google Colab:

On va mettre en place notre application et notre dashboard via Streamlit, sur lequel on va appliquer notre modèle LightGBM.


In [1]:
!pip install -q pyngrok

!pip install -q streamlit

!pip install -q streamlit_ace

     |████████████████████████████████| 745 kB 5.2 MB/s 
     |████████████████████████████████| 9.2 MB 4.8 MB/s 
     |████████████████████████████████| 4.7 MB 16.9 MB/s 
     |████████████████████████████████| 237 kB 47.0 MB/s 
     |████████████████████████████████| 182 kB 71.0 MB/s 
     |████████████████████████████████| 164 kB 50.3 MB/s 
     |████████████████████████████████| 78 kB 5.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 
     |████████████████████████████████| 3.6 MB 5.0 MB/s 


In [2]:
use_colab = True 

if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH ='/content/drive/MyDrive/'
else:
    PATH ='/data/'

Mounted at /content/drive


In [3]:
pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 5.3 MB/s 


In [4]:
import streamlit as st
import pandas as pd
import base64
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle as pkl
import shap
import streamlit.components.v1 as components


## Mise en place de l'application Streamlit : paramétrage


In [5]:
%%writefile app.py
import pandas as pd
import pickle
import streamlit as st
 
# loading the trained model
pickle_in = open("/content/drive/MyDrive/LightGBMmodel.pkl", "rb") 
model = pickle.load(pickle_in)
 
st.set_page_config(
    page_title="Loan Prediction App by Keyvan NOROOZI KIA",

)

st.set_option('deprecation.showPyplotGlobalUse', False)

######################
#main page layout
######################

st.title("Loan Default Prediction by Keyvan NOROOZI KIA")
st.subheader("Are you sure your loan applicant is surely going to pay the loan back?💸 "
                 "This machine learning app will help you to make a prediction to help you with your decision!")

col1, col2 = st.columns([1, 1])


with col2:
    st.write("""To borrow money, credit analysis is performed. Credit analysis involves the measure to investigate
the probability of the applicant to pay back the loan on time and predict its default/ failure to pay back.
These challenges get more complicated as the count of applications increases that are reviewed by loan officers.

Below you will see the app prediction and the probability(as a number) that customer will repay his loan.

If the customer can repay his loan, you will see a green text with a positive result. 
If the customer can't repay his loan, you will see a red text with a negative result.


""")

st.subheader("To predict default/ failure to pay back status, you need to follow the steps below:")
st.markdown("""
1. Enter/choose the parameters that best descibe your applicant on the left side bar;
2. Press the "Predict" button and wait for the result.
""")

st.subheader("Below you could find prediction result: ")

######################
#sidebar layout
######################

st.sidebar.title("Loan Applicant Info")

st.sidebar.write("Please choose parameters that descibe the applicant")

#input features
children = st.sidebar.slider("How many children?:",min_value=1, max_value=10,step=1)
loan_amnt =st.sidebar.slider("Please choose Loan amount you would like to apply:",min_value=1000, max_value=40000,step=500)
region_rating = st.sidebar.slider('Your region rating?', min_value =1, max_value=3, step=1 )
annual_inc =st.sidebar.slider("Please choose your annual income:", min_value=1000, max_value=200000,step=1000)
contract =st.sidebar.selectbox(('which type of loan do you want?'), ("cash","revolving"))
loan_previous =st.sidebar.slider("If you have a current loan and how much do you owe?:",min_value=0, max_value=40000,step=500)
delay =st.sidebar.slider(('how long (in days) was the delay between your payment and the payment date on your previous application ?'),min_value=0, max_value=4000,step=1)
requests= st.sidebar.slider("How many credit requests?:",min_value=0, max_value=25,step=1)


def preprocess(loan_amnt, children, region_rating, annual_inc, contract, loan_previous,delay,requests):
    # Pre-processing user input

    user_input_dict={'loan_amnt':[loan_amnt], 'children':[children], 'region_rating':[region_rating], 'annual_inc':[annual_inc], 'contract':[contract],
                'loan_previous':[loan_previous],'delay':[delay],'requests':[requests]}
    user_input=pd.DataFrame(data=user_input_dict)


    #user_input=np.array(user_input)
    #user_input=user_input.reshape(1,-1)

    cleaner_type = {"region_rating": {"1": 1.0, "2": 2.0,"3":3.0},
    "contract": {"cash": 1.0, "revolving": 0.0
    }
    }

    user_input = user_input.replace(cleaner_type)

    return user_input

#user_input=preprocess
user_input=preprocess(loan_amnt, children, region_rating, annual_inc, contract, loan_previous,delay,requests)

#predict button

btn_predict = st.sidebar.button("Predict")

if btn_predict:
    pred = model.predict_proba(user_input)[:, 1]


    if pred[0] < 0.5 :
        st.error('Warning! The applicant has a high risk to not pay the loan back!')
        st.text(pred[0])
    else:
        st.success('It is green! The aplicant has a high probability to pay the loan back!')
        st.text(pred[0])


Writing app.py


## Installation du tunnel local

In [6]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.057s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



## Exécution de Streamlit sur colab


In [7]:
!streamlit run /content/app.py &>/content/logs.txt &

## Mise en place du tunnel sur le port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 4.305s
your url is: https://lazy-crabs-guess-35-227-119-24.loca.lt


[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)